# Weather data visualization

The role of this notebook is to create a dashboard based on downloaded weather data for the 100 largest cities in Poland.

### Notebook configuration 

In [1]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output

### Loading the weather data file

In [2]:
full_weather_data = pd.read_csv('weather_data.csv')

### Creating time and date ranges

In [3]:
df_time = full_weather_data

df_time['Time'] = pd.to_datetime(df_time['Time'], format='%H:%M:%S')
df_time = df_time.sort_values(['Time'])

In [4]:
unique_dates = full_weather_data['Date'].unique()
unique_time = df_time['Time'].dt.strftime('%H:%M:%S').unique()

### dash configuration

In [5]:
app = dash.Dash(__name__)

In [6]:
app.layout = html.Div([
    html.Label('Select Info: '),
    dcc.Dropdown(
        id='info-dropdown',
        options=[
            {'label': 'Temperature [℃]', 'value': 'Temperature'},
            {'label': 'Pressure [hPa]', 'value': 'Pressure'},
            {'label': 'Humidity [%]', 'value': 'Humidity'},
            {'label': 'Clouds [%]', 'value': 'Clouds'},
            {'label': 'Wind Speed [km/h]', 'value': 'speed'},
            {'label': 'Rain [mm]', 'value': 'Rain'}
        ],
        value='Temperature',
        clearable=False
    ),
    
    html.Label('Select Time: '),
    dcc.Slider(
        id='time-slider',
        min=0,
        max=len(unique_time) - 1,
        step=1,
        value=3,
        marks={i: time for i, time in enumerate(unique_time)},
        included=False
    ),
    
    html.Label('Select Date:'),
    dcc.Slider(
        id='date-slider',
        min=0,
        max=len(unique_dates) - 1,
        step=1,
        value=2,
        marks={i: time for i, time in enumerate(unique_dates)},
        included=False
    ),
    
    dcc.Graph(id='weather-map')
])

In [7]:
@app.callback(
    Output('weather-map', 'figure'),
    [Input('info-dropdown', 'value'),
     Input('date-slider', 'value'),
     Input('time-slider', 'value')]
)

def update_map(selected_info, selected_date_index, selected_time_index):
    
    selected_date = unique_dates[selected_date_index]
    selected_time = unique_time[selected_time_index]
    
    filtered_df = full_weather_data[(full_weather_data['Date'] == selected_date) & (full_weather_data['Time'].dt.strftime('%H:%M:%S') == selected_time)]
    
    if filtered_df.empty:
        fig = {
            'data': [],
            'layout': {
                'xaxis': {'visible': False},
                'yaxis': {'visible': False},
                "annotations": [
                    {
                        'text': f'No data available for {selected_date} {selected_time}',
                        'xref': 'paper',
                        'yref': 'paper',
                        'showarrow': False,
                        'font': {'size': 20}
                    }
                ]
            }
        }
        return fig
    
    fig = px.density_mapbox(
        filtered_df,
        lat="width",
        lon="length",
        hover_name="City Name",
        custom_data=["City Name", selected_info, 'description'], 
        zoom=5,
        center={"lat": 52, "lon": 21}, 
        mapbox_style="stamen-terrain",
        z=selected_info
    )
    
    fig.update_traces(
        hovertemplate=(
        '<b>%{customdata[0]}</b><br>'  
        'Value: %{customdata[1]}<br>'
        'Weather in general: %{customdata[2]}<br>'  
        '<extra></extra>'
        )
    )
    
    fig.update_layout(mapbox_style='open-street-map')
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    
    return fig

### Launch of dashboard

In [8]:
if __name__ == '__main__':
    app.run_server(debug=True)